# Intraday Momentum Breakout Strategy: End-to-End Backtest

**ES & NQ E-mini Futures Momentum Trading System**

This notebook demonstrates the complete workflow for the Intraday Momentum Breakout Strategy:
1. Data Acquisition (ES & NQ Futures)
2. Noise Area Calculation (90-day volatility boundaries)
3. Signal Generation (Breakout detection)
4. Position Sizing (Volatility targeting: 3% daily)
5. Backtesting (With realistic transaction costs)
6. Performance Evaluation

---

## Key Strategy Parameters

- **Noise Area Lookback**: 90 days (optimized from research)
- **Target Volatility**: 3% daily portfolio volatility
- **Max Leverage**: 8x
- **Portfolio Allocation**: 50% NQ momentum, 25% ES momentum, 25% NQ long-only
- **Transaction Costs**: 1 tick slippage per side + $4.20 commission
- **Trading Hours**: 9:30 AM - 4:00 PM ET (intraday only)

---

## 1. Setup & Imports

Import all required modules and configure settings.

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

# Import strategy modules
from data_acquisition import FuturesDataDownloader
from noise_area import NoiseAreaCalculator, visualize_noise_area
from signal_generator import SignalGenerator
from position_sizer import PositionSizer
from backtester import Backtester
from performance_evaluator import PerformanceEvaluator, visualize_performance

# Standard libraries
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ All modules imported successfully")
print(f"Current directory: {os.getcwd()}")

## 2. Load Configuration

Load strategy parameters from config file.

In [ ]:
# Load configuration
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Display key parameters
print("="*60)
print("STRATEGY CONFIGURATION")
print("="*60)
print(f"\nNoise Area:")
print(f"  Lookback: {config['strategy']['noise_area']['lookback_days']} days")
print(f"  Method: {config['strategy']['noise_area']['method']}")
print(f"  Upper percentile: {config['strategy']['noise_area']['upper_percentile']}%")
print(f"  Lower percentile: {config['strategy']['noise_area']['lower_percentile']}%")

print(f"\nPosition Sizing:")
print(f"  Target volatility: {config['strategy']['position_sizing']['target_daily_volatility']}%")
print(f"  Max leverage: {config['strategy']['position_sizing']['max_leverage']}x")
print(f"  Volatility estimation: {config['strategy']['position_sizing']['volatility_estimation']}")

print(f"\nPortfolio Allocation:")
for strategy, allocation in config['strategy']['portfolio']['allocation'].items():
    print(f"  {strategy}: {allocation}%")

print(f"\nTransaction Costs:")
print(f"  Commission: ${config['strategy']['transaction_costs']['commission_per_contract']:.2f} per contract")
print(f"  Slippage: {config['strategy']['transaction_costs']['slippage_ticks']} ticks per side")
print(f"  ES tick value: ${config['strategy']['transaction_costs']['ES']['tick_value']:.2f}")
print(f"  NQ tick value: ${config['strategy']['transaction_costs']['NQ']['tick_value']:.2f}")

print(f"\nBacktest Period:")
print(f"  Start: {config['data']['start_date']}")
print(f"  End: {config['data']['end_date']}")
print(f"  Initial Capital: ${config['strategy']['portfolio']['initial_capital']:,}")
print("="*60)

## 3. Data Acquisition

Download ES & NQ futures data. For testing, synthetic data is generated if real data is unavailable.

In [ ]:
# Create data directory
os.makedirs('data', exist_ok=True)
os.makedirs('results', exist_ok=True)

# Initialize downloader
downloader = FuturesDataDownloader(config)

# Download data (or use cached)
try:
    data = downloader.load_data('data')
    if len(data) == 2:
        print("✓ Loaded cached data")
    else:
        raise FileNotFoundError
except:
    print("Downloading new data...")
    data = downloader.download_all_data()
    downloader.save_data(data, 'data')

es_data = data['ES'].copy()
nq_data = data['NQ'].copy()

print(f"\n✓ Data loaded successfully")
print(f"  ES: {len(es_data)} bars from {es_data.index[0]} to {es_data.index[-1]}")
print(f"  NQ: {len(nq_data)} bars from {nq_data.index[0]} to {nq_data.index[-1]}")

### Visualize Raw Data

In [ ]:
# Plot raw price data
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# ES
ax = axes[0]
ax.plot(es_data.index, es_data['Close'], linewidth=1.5, label='ES Close', color='blue')
ax.fill_between(es_data.index, es_data['Low'], es_data['High'], alpha=0.2, color='blue')
ax.set_title('ES E-mini S&P 500 Futures - Price Action', fontsize=14, fontweight='bold')
ax.set_ylabel('Price')
ax.legend()
ax.grid(True, alpha=0.3)

# NQ
ax = axes[1]
ax.plot(nq_data.index, nq_data['Close'], linewidth=1.5, label='NQ Close', color='orange')
ax.fill_between(nq_data.index, nq_data['Low'], nq_data['High'], alpha=0.2, color='orange')
ax.set_title('NQ E-mini Nasdaq-100 Futures - Price Action', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Price')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/01_raw_price_data.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Raw data visualization saved")

## 4. Noise Area Calculation

Calculate volatility-based boundaries that define "normal" price fluctuation.

In [ ]:
# Initialize calculator
calculator = NoiseAreaCalculator(config)

# Calculate noise area for ES
print("\nCalculating noise area for ES...")
es_data = calculator.calculate_noise_area(es_data)
es_data = calculator.identify_breakouts(es_data)

# Calculate noise area for NQ
print("\nCalculating noise area for NQ...")
nq_data = calculator.calculate_noise_area(nq_data)
nq_data = calculator.identify_breakouts(nq_data)

print("\n✓ Noise area calculation complete")

### Visualize Noise Area & Breakouts

In [ ]:
# Visualize ES noise area
visualize_noise_area(es_data, 'ES', start_idx=0, end_idx=min(1000, len(es_data)))

# Visualize NQ noise area
visualize_noise_area(nq_data, 'NQ', start_idx=0, end_idx=min(1000, len(nq_data)))

print("✓ Noise area visualizations created")

### Breakout Statistics

In [ ]:
# ES breakout stats
print("="*60)
print("ES BREAKOUT STATISTICS")
print("="*60)
print(f"Total bars: {len(es_data)}")
print(f"Break above: {es_data['break_above'].sum()} bars ({es_data['break_above'].mean()*100:.2f}%)")
print(f"Break below: {es_data['break_below'].sum()} bars ({es_data['break_below'].mean()*100:.2f}%)")
print(f"Inside noise: {es_data['inside_noise'].sum()} bars ({es_data['inside_noise'].mean()*100:.2f}%)")
print(f"Momentum failures: {es_data['momentum_failure'].sum()}")

print("\n" + "="*60)
print("NQ BREAKOUT STATISTICS")
print("="*60)
print(f"Total bars: {len(nq_data)}")
print(f"Break above: {nq_data['break_above'].sum()} bars ({nq_data['break_above'].mean()*100:.2f}%)")
print(f"Break below: {nq_data['break_below'].sum()} bars ({nq_data['break_below'].mean()*100:.2f}%)")
print(f"Inside noise: {nq_data['inside_noise'].sum()} bars ({nq_data['inside_noise'].mean()*100:.2f}%)")
print(f"Momentum failures: {nq_data['momentum_failure'].sum()}")

## 5. Signal Generation

Generate trading signals based on breakout detection with confirmation and volume filters.

In [ ]:
# Initialize signal generator
signal_gen = SignalGenerator(config)

# Generate signals for ES
print("\nGenerating signals for ES...")
es_data = signal_gen.generate_signals(es_data)

# Generate signals for NQ
print("\nGenerating signals for NQ...")
nq_data = signal_gen.generate_signals(nq_data)

print("\n✓ Signal generation complete")

### Signal Analysis

In [ ]:
# ES signal stats
es_entries = es_data[es_data['entry_signal']]
es_exits = es_data[es_data['exit_signal']]

print("="*60)
print("ES SIGNAL STATISTICS")
print("="*60)
print(f"Entry signals: {len(es_entries)}")
print(f"  Long: {(es_entries['signal'] == 1).sum()}")
print(f"  Short: {(es_entries['signal'] == -1).sum()}")
print(f"Exit signals: {len(es_exits)}")
if len(es_exits) > 0:
    print(f"  Exit reasons:")
    for reason, count in es_exits['exit_reason'].value_counts().items():
        print(f"    {reason}: {count}")
print(f"Avg signal strength: {es_entries['signal_strength'].mean():.1f}")

# NQ signal stats
nq_entries = nq_data[nq_data['entry_signal']]
nq_exits = nq_data[nq_data['exit_signal']]

print("\n" + "="*60)
print("NQ SIGNAL STATISTICS")
print("="*60)
print(f"Entry signals: {len(nq_entries)}")
print(f"  Long: {(nq_entries['signal'] == 1).sum()}")
print(f"  Short: {(nq_entries['signal'] == -1).sum()}")
print(f"Exit signals: {len(nq_exits)}")
if len(nq_exits) > 0:
    print(f"  Exit reasons:")
    for reason, count in nq_exits['exit_reason'].value_counts().items():
        print(f"    {reason}: {count}")
print(f"Avg signal strength: {nq_entries['signal_strength'].mean():.1f}")

### Visualize Signals

In [ ]:
# Plot signals on price chart
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# ES signals
ax = axes[0]
plot_data = es_data.iloc[:min(1000, len(es_data))]
ax.plot(plot_data.index, plot_data['Close'], linewidth=1.5, color='black', label='ES Close')
ax.plot(plot_data.index, plot_data['upper_boundary'], linestyle='--', color='red', alpha=0.5, label='Upper Boundary')
ax.plot(plot_data.index, plot_data['lower_boundary'], linestyle='--', color='green', alpha=0.5, label='Lower Boundary')

# Mark entries
long_entries = plot_data[plot_data['entry_signal'] & (plot_data['signal'] == 1)]
short_entries = plot_data[plot_data['entry_signal'] & (plot_data['signal'] == -1)]
ax.scatter(long_entries.index, long_entries['Close'], marker='^', s=100, color='green', label='Long Entry', zorder=5)
ax.scatter(short_entries.index, short_entries['Close'], marker='v', s=100, color='red', label='Short Entry', zorder=5)

ax.set_title('ES Signals with Noise Area', fontsize=14, fontweight='bold')
ax.set_ylabel('Price')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

# NQ signals
ax = axes[1]
plot_data = nq_data.iloc[:min(1000, len(nq_data))]
ax.plot(plot_data.index, plot_data['Close'], linewidth=1.5, color='black', label='NQ Close')
ax.plot(plot_data.index, plot_data['upper_boundary'], linestyle='--', color='red', alpha=0.5, label='Upper Boundary')
ax.plot(plot_data.index, plot_data['lower_boundary'], linestyle='--', color='green', alpha=0.5, label='Lower Boundary')

# Mark entries
long_entries = plot_data[plot_data['entry_signal'] & (plot_data['signal'] == 1)]
short_entries = plot_data[plot_data['entry_signal'] & (plot_data['signal'] == -1)]
ax.scatter(long_entries.index, long_entries['Close'], marker='^', s=100, color='green', label='Long Entry', zorder=5)
ax.scatter(short_entries.index, short_entries['Close'], marker='v', s=100, color='red', label='Short Entry', zorder=5)

ax.set_title('NQ Signals with Noise Area', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Price')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/02_signals_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Signal visualizations created")

## 6. Position Sizing

Calculate position sizes using volatility targeting to maintain 3% daily portfolio volatility.

In [ ]:
# Initialize position sizer
sizer = PositionSizer(config)

# Calculate positions for portfolio
portfolio = sizer.calculate_portfolio_positions(es_data, nq_data)

print("\n✓ Position sizing complete")

### Position Size Analysis

In [ ]:
# Analyze position sizes
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# ES momentum
ax = axes[0]
plot_data = portfolio['ES_momentum'].iloc[:min(2000, len(portfolio['ES_momentum']))]
ax.plot(plot_data.index, plot_data['position_size'], linewidth=1, label='Position Size (contracts)', color='blue')
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.fill_between(plot_data.index, 0, plot_data['position_size'], alpha=0.3, color='blue')
ax.set_title('ES Momentum - Position Sizes', fontsize=12, fontweight='bold')
ax.set_ylabel('Contracts')
ax.legend()
ax.grid(True, alpha=0.3)

# NQ momentum
ax = axes[1]
plot_data = portfolio['NQ_momentum'].iloc[:min(2000, len(portfolio['NQ_momentum']))]
ax.plot(plot_data.index, plot_data['position_size'], linewidth=1, label='Position Size (contracts)', color='orange')
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.fill_between(plot_data.index, 0, plot_data['position_size'], alpha=0.3, color='orange')
ax.set_title('NQ Momentum - Position Sizes', fontsize=12, fontweight='bold')
ax.set_ylabel('Contracts')
ax.legend()
ax.grid(True, alpha=0.3)

# NQ long-only
ax = axes[2]
plot_data = portfolio['NQ_long_only'].iloc[:min(2000, len(portfolio['NQ_long_only']))]
ax.plot(plot_data.index, plot_data['position_size'], linewidth=1, label='Position Size (contracts)', color='green')
ax.fill_between(plot_data.index, 0, plot_data['position_size'], alpha=0.3, color='green')
ax.set_title('NQ Long-Only - Position Sizes', fontsize=12, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Contracts')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/03_position_sizes.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Position size visualization created")

## 7. Backtesting

Run event-driven backtest with realistic transaction costs (1 tick slippage + commission).

In [ ]:
# Initialize backtester
backtester = Backtester(config)

# Run backtest
equity_curve = backtester.run_backtest(portfolio)
trades_df = backtester.get_trades_dataframe()

print("\n✓ Backtesting complete")

### Quick Equity Curve Preview

In [ ]:
# Plot equity curve
plt.figure(figsize=(16, 6))
plt.plot(equity_curve.index, equity_curve['portfolio_value'], linewidth=2, color='blue')
plt.axhline(y=config['strategy']['portfolio']['initial_capital'], color='red', 
            linestyle='--', alpha=0.5, label='Initial Capital')
plt.title('Equity Curve', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('results/04_equity_curve_preview.png', dpi=150, bbox_inches='tight')
plt.show()

# Quick stats
total_return = (equity_curve['portfolio_value'].iloc[-1] / equity_curve['portfolio_value'].iloc[0] - 1) * 100
print(f"\nTotal Return: {total_return:.2f}%")
print(f"Final Portfolio Value: ${equity_curve['portfolio_value'].iloc[-1]:,.0f}")
print(f"Total Trades: {len(trades_df)}")

## 8. Performance Evaluation

Calculate comprehensive performance metrics and generate visualizations.

In [ ]:
# Initialize evaluator
evaluator = PerformanceEvaluator(config)

# Evaluate performance
metrics = evaluator.evaluate_strategy(equity_curve, trades_df)

print("\n✓ Performance evaluation complete")

### Comprehensive Visualizations

In [ ]:
# Generate full performance visualization
visualize_performance(equity_curve, trades_df, metrics)

## 9. Detailed Trade Analysis

In [ ]:
if len(trades_df) > 0:
    print("="*60)
    print("TRADE ANALYSIS")
    print("="*60)
    
    # By symbol
    print("\nTrades by Symbol:")
    print(trades_df['symbol'].value_counts())
    
    # By side
    print("\nTrades by Side:")
    print(trades_df['side'].value_counts())
    
    # By exit reason
    print("\nExit Reasons:")
    print(trades_df['exit_reason'].value_counts())
    
    # Top 10 winning trades
    print("\nTop 10 Winning Trades:")
    top_winners = trades_df.nlargest(10, 'pnl_net')[['timestamp', 'symbol', 'side', 'pnl_net', 'holding_bars']]
    print(top_winners.to_string(index=False))
    
    # Top 10 losing trades
    print("\nTop 10 Losing Trades:")
    top_losers = trades_df.nsmallest(10, 'pnl_net')[['timestamp', 'symbol', 'side', 'pnl_net', 'holding_bars']]
    print(top_losers.to_string(index=False))
    
    # Distribution of holding periods
    plt.figure(figsize=(12, 5))
    plt.hist(trades_df['holding_bars'], bins=50, alpha=0.7, color='blue', edgecolor='black')
    plt.axvline(x=trades_df['holding_bars'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Mean: {trades_df["holding_bars"].mean():.1f} bars')
    plt.title('Distribution of Holding Periods', fontsize=14, fontweight='bold')
    plt.xlabel('Holding Bars (5-min)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('results/05_holding_periods.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("No trades executed")

## 10. Save Results

In [ ]:
# Save all results
portfolio['ES_momentum'].to_csv('results/es_momentum_data.csv')
portfolio['NQ_momentum'].to_csv('results/nq_momentum_data.csv')
portfolio['NQ_long_only'].to_csv('results/nq_long_only_data.csv')
equity_curve.to_csv('results/equity_curve.csv')
trades_df.to_csv('results/trades.csv')

# Save metrics
metrics_df = pd.DataFrame([metrics]).T
metrics_df.columns = ['Value']
metrics_df.to_csv('results/performance_metrics.csv')

print("✓ All results saved to results/ directory")
print("\nFiles created:")
print("  - es_momentum_data.csv")
print("  - nq_momentum_data.csv")
print("  - nq_long_only_data.csv")
print("  - equity_curve.csv")
print("  - trades.csv")
print("  - performance_metrics.csv")
print("  - Various PNG visualizations")

## Summary

### Key Results

This notebook demonstrated the complete end-to-end workflow for the Intraday Momentum Breakout Strategy on ES & NQ futures.

### Strategy Highlights
- ✓ Volatility-based noise area calculation (90-day lookback)
- ✓ Breakout detection with confirmation and volume filters
- ✓ Volatility-targeted position sizing (3% daily target)
- ✓ Intraday-only execution (no overnight risk)
- ✓ Conservative transaction cost modeling (1 tick slippage per side)
- ✓ Portfolio diversification (50/25/25 allocation)

### Performance Metrics Summary

Check the `results/performance_metrics.csv` file for detailed metrics including:
- Sharpe Ratio (primary optimization target)
- Maximum Drawdown
- Win Rate & Profit Factor
- Sortino & Calmar Ratios
- Transaction Cost Analysis

### Next Steps

1. **Walk-Forward Optimization**: Run `02_walk_forward_optimization.ipynb`
2. **Parameter Sensitivity**: Run `03_parameter_sensitivity_analysis.ipynb`
3. **Stress Testing**: Test extreme market conditions
4. **Live Trading Preparation**: Setup real-time data feeds and OMS integration

---

**Note**: This is a research/educational implementation. Not financial advice. Past performance does not guarantee future results.